# Generate Workout Plan

## Recommend exercises

In [1]:
import pandas as pd
import numpy as np

# Import data generated upstream
exercises_data_full = pd.read_csv("../app/data/exercises_difficulty_classification_full.csv")
merged_data = pd.read_csv("../app/data/merged_exercise_user_data.csv")
cosine_similarity_train = np.load("../app/data/transformed/cosine_similarity_train.npy")

In [2]:
# Recommend top k similar items for a given item
def recommend_similar_items(item_index, item_name, threshold = 0.65, num_top = 3):
    # Use cosine similarity for recommendations
    similarities = cosine_similarity_train[item_index]

    # Get indices and scores of items with similarity >= threshold
    similar_indices_and_scores = [(idx, score) for idx, score in enumerate(similarities) if score >= threshold]

    # Convert to DataFrame
    result_df = pd.DataFrame(similar_indices_and_scores, columns = ['index', 'Cosine_Similarity_Score'])
    result_df['Compared_Exercise'] = item_name 

    return result_df.sort_values('Cosine_Similarity_Score', ascending = False).reset_index(drop = True).head(num_top)

# Get exercises matching user inputs
def get_exercises_from_user_inputs(experience, muscle, type, equipment):
    has_experience = merged_data.Fitness_Experience == experience
    has_muscle = merged_data.Desired_Muscle_Groups == muscle
    has_type = merged_data.Workout_Type == type
    has_equipment = merged_data.Available_Equipment == equipment
    
    filtered_df = merged_data[has_experience & has_muscle & has_type & has_equipment]
    filtered_df = filtered_df.drop(['User_ID', 'Workout_Frequency'], axis = 1).drop_duplicates()

    # Creating a new dataframe with the specified columns
    df_selected_columns = filtered_df[['index', 'Name', 'Rating']]

    return df_selected_columns

# Get data of recommended exercises from user inputs
def get_exercise_recommendation(experience, muscle, type, equipment):
    exercise_queries = get_exercises_from_user_inputs(experience, muscle, type, equipment)

    ret_list = []
    for row in exercise_queries.iterrows():
        df_recommend = recommend_similar_items(row[1]['index'], row[1].Name)
        df_recommend_full = pd.merge(df_recommend, exercises_data_full, on = 'index')
        ret_list.append(df_recommend_full)

    ret = pd.concat(ret_list, ignore_index = True).rename({'Name': 'Recommended_Exercise'}, axis = 1)
    ret = ret.drop_duplicates(subset = 'index', keep = 'first') # remove duplicated exercises
    
    return ret[['Recommended_Exercise','Rating']].sort_values('Rating', ascending = False).head(100)
    

In [3]:
# Example usage/user inputs
experience = 'Intermediate'
muscle = 'Chest, Legs'
type = 'Strength'
equipment = 'Gym (Machine)'
frequency = "7 days/week"
split = "Upper/Lower Split"
repeat = "50%"

sample_recommend = get_exercise_recommendation(experience, muscle, type, equipment)

In [4]:
sample_recommend

,Recommended_Exercise,Rating
18,Low-cable cross-over,9.1
0,Chest dip,9.0
47,Weighted Jump Squat,9.0
21,Incline cable chest fly,9.0
62,Dumbbell squat,8.9
46,Barbell front squat,8.9
44,Smith machine box squat,8.8
38,Machine Squat,8.8
36,Narrow-stance leg press,8.8
19,Cable Crossover,8.8


## Integrate OpenAI API

In [5]:
import openai
from openai import OpenAI
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv()

def generate_chat_completion(content):    
    try:
        client = OpenAI(api_key = os.environ.get("OPENAI_API_KEY"))
        completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": content,
            }
        ],
        model = "gpt-4", # replace with desired model
        )
        return completion.choices[0].message.content

    except openai.APIConnectionError as e:
        print("The server could not be reached")
        print(e.__cause__)  # an underlying Exception, likely raised within httpx.
    except openai.RateLimitError as e:
        print("A 429 status code was received; we should back off a bit.")
    except openai.APIStatusError as e:
        print("Another non-200-range status code was received")
        print(e.status_code)
        print(e.response)

In [6]:
def get_split_info(split):
    if split == 'Full Body Workout':
        return "Ensure the exercises target all muscle groups for every workout day."
    elif split == 'Upper/Lower Split':
        text = "Ensure upper body exercises and lower body exercises are trained on separate days. Do not combine upper body and lower body exercises for a given day. "
    elif split == 'Push/Pull Split':
        text = "Ensure push exercises and pull exercises are trained on separate days. Do not combine push and pull exercises for a given day. "
    elif split == 'Push/Pull/Legs Split':
        text = "Ensure push exercises, pull exercises and leg exercises. Do not combine push, pull or legs exercises for a given day. "
    elif split == 'Bro Split':
        text = "Ensure each individual muscle group is trained on a separate day. Do not combine multiple major muscle groups for a given day. "
    else:
        return "Determine the best workout plan given the list of exercises, either separating the days by muscle group or a combination each day."

    return text + "Avoid consecutive days that train the same split."

In [7]:
input = f"""
1) Formulate a one-week workout schedule following a {split} plan, where each day should consist of exactly four exercises. The schedule must include exercises for {frequency}.
2) {get_split_info(split)}
3) Choose exclusively {equipment} exercises and exercises categorized as {type} type and of {experience} difficulty. The exercise should focus on {muscle} muscle engagement. Exercises may include compound exercises if multiple muscles groups are specified.
4) Allow for up to {repeat} repitition of exercises. However, no two days are exactly the same, unless in cases of 100% repetition. While for 0% repetition, select entirely different exercises with zero repeats. Prioritize highly rated exercises to repeat.
5) Ensure all individual muscles of each muscle group provided are covered by the end of the workout schedule.
6) The database of exercises to choose from is provided below in text format enclosed in quotations. The table includes the exercise name and rating as columns. Avoid selecting exercises with a rating of '0.0', only selecting them when options are sparse.
   "{sample_recommend.to_string()}"
7) The output format should strictly follow the structure provided below delimited by triple backticks. All brackets are placeholders and should be replaced with the specified information. If a day is dedicated as a rest day, put 'Rest Day' as the exercise name. There are no rest days for a '7 days/week' schedule.
```
Day 1:
    Exercise 1: [Exercise 1 Name]
    Exercise 2: [Exercise 2 Name]
    Exercise 3: [Exercise 3 Name]
    Exercise 4: [Exercise 4 Name]
    Rationale: [Explain the specific muscle groups targeted and how the exercises fit into the overall {split} plan.]

Day 2:
    [Repeat the structure for Day [1] with a different set of exercises]

 ...

Day 7:
    [Repeat the structure for Day [1] with a different set of exercises]
```
"""

In [8]:
response = generate_chat_completion(input)

In [9]:
response

'Given your requirements, your schedule could look like the following:\n\n```\nDay 1 (Upper Body):\n    Exercise 1: Chest dip\n    Exercise 2: Incline cable chest fly\n    Exercise 3: Cable Crossover\n    Exercise 4: Low-cable cross-over\n    Rationale: Focused primarily on the chest area, incorporating both heavy movements with dips for muscle mass and incline cable chest fly for range of motion. Crossover exercises are intended to isolate the pecs from different angles.\n\nDay 2 (Lower Body): \n    Exercise 1: Weighted Jump Squat\n    Exercise 2: Barbell front squat\n    Exercise 3: Machine Squat\n    Exercise 4: Narrow-stance leg press\n    Rationale: Concentrated on legs, including quadriceps and hamstrings. Squat variations develop power and strength while the leg press assists in isolating muscles and providing shape.\n\nDay 3 (Upper Body):\n    Exercise 1: Butterfly\n    Exercise 2: Machine chest press\n    Exercise 3: Smith machine bench press\n    Exercise 4: Leverage Chest Pr